In [1]:
import os
import time
import imageio
import json
import glob
from datetime import datetime, timedelta

import folium 
import geojsoncontour
import numpy as np

from folium import plugins
from netCDF4 import Dataset as NetCDFFile
import matplotlib.pyplot as plt

import branca.colormap as cm
from matplotlib import colors as mcolors
from wrf import getvar, interplevel, to_np, latlon_coords

from PIL import Image
from pathlib import Path

from selenium import webdriver

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

In [3]:
nc_file = NetCDFFile('../wrf_output')
time_size = nc_file.dimensions['Time'].size

In [4]:
height = getvar(nc_file, 'height', timeidx=1)
Var_all = getvar(nc_file, 'tc', timeidx=1)

var = interplevel(Var_all, height, 200)

print(var)

(lats, lons) = latlon_coords(var)

<xarray.DataArray 'temp_interp' (south_north: 81, west_east: 90)>
array([[27.181078, 27.1728  , 27.28623 , ..., 26.542267, 26.527937,
        26.513374],
       [27.066309, 26.896858, 26.981695, ..., 26.524864, 26.506876,
        26.487972],
       [27.003468, 26.870806, 26.908741, ..., 26.499397, 26.480488,
        26.462528],
       ...,
       [24.250288, 24.259905, 24.264938, ..., 24.66926 , 24.664595,
        24.658937],
       [24.250658, 24.258316, 24.26341 , ..., 24.65932 , 24.6513  ,
        24.645208],
       [24.25143 , 24.256514, 24.26082 , ..., 24.64702 , 24.639523,
        24.631733]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 -74.96 -74.96 ... -74.68 -74.68
    XLAT     (south_north, west_east) float32 10.85 10.85 10.85 ... 11.11 11.11
    XTIME    float32 180.0
    Time     datetime64[ns] 2022-03-27T21:00:00
    level    int64 200
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:      104
    units:        

In [6]:
def get_data(nc_file: NetCDFFile, timeidx:int):
    dx = 100
    height = getvar(nc_file, 'height', timeidx=timeidx)

    u_all = getvar(nc_file, 'ua', timeidx=timeidx)
    v_all = getvar(nc_file, 'va', timeidx=timeidx)
    w_all = getvar(nc_file, 'wa', timeidx=timeidx)
    T_all = getvar(nc_file, 'tc', timeidx=timeidx)
    P_all = getvar(nc_file, 'pressure', timeidx=timeidx)

    P = interplevel(P_all, height, dx)
    T = interplevel(T_all, height, dx)
    u = interplevel(u_all, height, dx)
    v = interplevel(v_all, height, dx)
    w = interplevel(w_all, height, dx)
    
    data = {
        'wind': ('Wind velocity in m/s', np.sqrt(u ** 2 + v ** 2 + w ** 2)),
        'temp': ('Tempeture in celsius', T),
        'uwind': ('U_Wind Velocity in m/s', u),
        'vwind': ('V_Wind Velocity in m/s', v),
        'press': ('Pressure in hPa', P)
    }

    return data


def get_folium(nc_file: NetCDFFile, timeidx: int, nc_var: str, start_date: datetime):
    data = get_data(nc_file, timeidx)
    
    (caption, variable) = data[nc_var]
    
    (lats, lons) = latlon_coords(variable)

    figure = plt.figure()
    ax = figure.add_subplot(111)

    contour = ax.contourf(lons, lats, variable, cmap=plt.cm.jet)
    cbar = figure.colorbar(contour)

    gj = json.loads(geojsoncontour.contourf_to_geojson(contourf=contour, ndigits=6, unit='m'))

    folium_map = folium.Map(
        location=[lats.mean(), lons.mean()],
        tiles='Cartodb Positron',
        zoom_start=11,
        zoom_control=False,
        scrollWheelZoom=False,
        dragging=False
    )

    folium.GeoJson(
        gj,
        style_function=lambda x: {
            'color': x['properties']['stroke'],
            'weight': x['properties']['stroke-width'],
            'fillColor': x['properties']['fill'],
            'opacity': 0.4,
        },
        name='geojson'
    ).add_to(folium_map)

    colormap = cm.LinearColormap(
        colors=['darkblue', 'blue', 'cyan', 'green', 'greenyellow', 'yellow', 'orange', 'red', 'darkred'],
        index=np.array(cbar.values), vmin=cbar.values[0],
        vmax=cbar.values[len(cbar.values) - 1],
        caption=caption
    )

    folium_map.add_child(colormap)

    date = start_date + timedelta(hours=timeidx * 3) - timedelta(hours=5)
    folium_map.get_root().html.add_child(folium.Element('<span style="position:fixed;z-index:999;font-size:12px;margin:4px">{}</span>'.format(date)))

    return (folium_map, figure)


def get_image(timeidx: int, nc_var: str, start_date: datetime):
    (f_map, fig) = get_folium(nc_file, timeidx, nc_var, start_date)

    html_file = f"{nc_var}_{timeidx}.html"
    png_file = f"{nc_var}_{timeidx}.png"

    f_map.save(html_file)

    driver.set_window_size(600, 600)
    driver.get(f"file://{os.getcwd()}/{html_file}")

    time.sleep(1)
    driver.save_screenshot(png_file)

    img = imageio.imread(png_file)

    os.remove(html_file)
    os.remove(png_file)

    return img

In [7]:
nc_var = 'temp'
start_date = datetime.strptime("2022-03-26 18", '%Y-%m-%d %H')

results = [get_image(timeidx, nc_var, start_date) for timeidx in range(time_size)]

imageio.mimwrite(f"{nc_var}_test.gif", results, fps=1);